# Sentiment Analysis

## Helper functions

### Remove apostrophies from negotiations (don't, haven't, ...)

In [ ]:
def write_out_negotiations(text):
    
    # haven't, hasn't

    text = re.sub("haven't", "have not", text)
    text = re.sub("Haven't", "Have not", text)
    text = re.sub("hasn't", "has not", text)
    text = re.sub("Hasn't", "Has not", text)

    ## isn't, can't, don't

    text = re.sub("mustn't", "must not", text)
    text = re.sub("Mustn't", "Must not", text)

    text = re.sub("can'", "can not", text)
    text = re.sub("Can't", "Can not", text)

    text = re.sub("isn't", "is not", text)
    text = re.sub("Isn't", "Is not", text)

    text = re.sub("aren't", "are not", text)
    text = re.sub("Aren't", "Are not", text)

    text = re.sub("don't", "do not", text)
    text = re.sub("Don't", "Do not", text)

    text = re.sub("doesn't", "does not", text)
    text = re.sub("doesn't", "does not", text)

    text = re.sub("didn't", "did not", text)
    text = re.sub("Didn't", "Did not", text)

    ## wasn't, won't

    text = re.sub("wasn't", "was not", text)
    text = re.sub("Wasn't", "Was not", text)

    text = re.sub("won't", "will not", text)
    text = re.sub("Won't", "Will not", text)

    ## wouldn't, couldn't

    text = re.sub("couldn't", "could not", text)
    text = re.sub("shouldn't", "should not", text)
    return text